In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.svm import OneClassSVM 
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from collections import defaultdict
import itertools
import pickle

In [2]:
df = pd.read_csv("../data/run-over-dataset.csv")
print(df.shape)

columns_to_drop = ['VERBALE', 'DATA', 'Tot Testa', 'Tot Torace', 'Tot Addome', 'Tot Scheletro',
                    'Totale', 'Tot Volta cranica', 'Tot Base cranica', 
                    'Tot Neuroc.', 'Tot Splancnoc.', 'Tot Testa',
                    'Tot Tratto toracico', 'Tot Tratto lombare', 'Tot Rachide',
                    ' Totale coste', 'Sterno in toto', 'Tot Bacino', 'I costa dx', 'II costa dx',
                    'III costa dx', 'IV costa dx', 'V costa dx', 'VI costa dx', 'VII costa dx', 
                    'VIII costa dx', 'IX costa dx', 'X costa dx', 'XI costa dx', 'XII costa dx',
                    'I costa sx', 'II costa sx', 'III costa sx', 'IV costa sx', 'V costa sx', 
                    'VI costa sx', 'VII costa sx', 'VIII costa sx', 'IX costa sx', 
                    'X costa sx', 'XI costa sx', 'XII costa sx']

X = df.drop(columns=columns_to_drop)
print(X.shape)

X['ALTEZZA'] = [int(float(h.replace(',', '.'))*100) for h in X['ALTEZZA']]
X['PESO'] = [int(float(str(h).replace(',', '.'))) for h in X['PESO']]
X['BMI'] = [float(str(h).replace(',', '.')) for h in X['BMI']]

num_unique_values = X.nunique()
constant_columns = num_unique_values[num_unique_values == 1].index.tolist()

X = X.drop(columns=constant_columns)
X = X.T.drop_duplicates().T
print(X.shape)

(130, 367)
(130, 326)
(130, 274)


In [3]:
pca = PCA(n_components=0.95, svd_solver='full')
X_reduced = pca.fit_transform(X)
X_reduced.shape


(130, 5)

In [4]:

total_variance = 0
columns_variance = {}
n_cols = X.shape[1]

for col in X.columns:
    columns_variance[col] = X[col].var()
    total_variance += X[col].var()

current_variance = total_variance
for idx, (col, var) in enumerate(sorted(columns_variance.items(), key=lambda item: item[1])):
    current_variance -= var 
    if (current_variance / total_variance <= 0.95):
        print(n_cols - (idx + 1))
        break

16


In [5]:
def reduce_cols(X, components):
    total_variance = 0
    columns_variance = {}
    n_cols = X.shape[1]

    for col in X.columns:
        columns_variance[col] = X[col].var()
        total_variance += X[col].var()

    cols_to_drop = []
    for idx, (col, var) in enumerate(sorted(columns_variance.items(), key=lambda item: item[1])):
        if (n_cols - (idx + 1) >= components):
            cols_to_drop.append(col) 
        else:
            break

    return X.drop(columns=cols_to_drop)

new_X = reduce_cols(X, 250)
new_X.shape

(130, 250)

In [12]:
random_seeds = np.random.randint(2343, 3485327, size=10)
random_seeds

array([2680935,  475572, 2129650, 3270809,  324016, 2761438, 1164095,
       2254681, 3166265,  759491])

In [13]:
def nested_cv(X, components, random_seeds, mod_selection_score=accuracy_score):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    best_overall_accuracy = 0
    best_overall_params = {'kernel': '', 'gamma': 0, 'nu': 0}

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo')

    X = reduce_cols(X, components)
    X = X.values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

            best_score = 0
            for trainval_idx, valid_idx in inner_cv.split(X_train, y_train):
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                idxs_neg = np.where(y_trainval == 1)[0]
                X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
                y_valid = np.append(y_valid, y_trainval[idxs_neg])

                X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                for params in itertools.product(kernels, gammas, nus):
                    clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                    clf.fit(X_trainval)

                    pred_values = clf.predict(X_valid)
                    true_values = [1 if y == 0 else -1 for y in y_valid]

                    score = 0
                    if (mod_selection_score == precision_score):
                        score = mod_selection_score(true_values, pred_values, zero_division=0.0)
                    else:
                        score = mod_selection_score(true_values, pred_values)
                        
                    if score > best_score:
                        best_score = score
                        best_params['kernel'] = params[0]
                        best_params['gamma'] = params[1]
                        best_params['nu'] = params[2]

            clf = OneClassSVM(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            if accuracy_score(true_values, pred_values) > best_overall_accuracy:
                best_overall_accuracy = accuracy_score(true_values, pred_values)
                best_overall_params['kernel'] = best_params['kernel']
                best_overall_params['gamma'] = best_params['gamma']
                best_overall_params['nu'] = best_params['nu']


            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_overall_params['kernel'],
            'best gamma': best_overall_params['gamma'],
            'best nu': best_overall_params['nu'],
            'score used for model selection': mod_selection_score.__name__,
            'method used for model selection': 'nested cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'best overall accuracy': best_overall_accuracy * 100,
            'components': components}

In [14]:
results = nested_cv(X, 250, random_seeds, accuracy_score)
scores_df = pd.DataFrame(results, index=[0])
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,best overall accuracy,components
0,OneClassSVM,rbf,1.000000e-09,0.13,accuracy_score,nested cv,57.615385,10.810087,64.475,31.888391,33.428571,25.92926,80.769231,250


In [15]:
def add_record(df, record):
    new_record = pd.DataFrame(record, index=[0])
    df = pd.concat([df, new_record], ignore_index=True)
    return df  

In [16]:
scores_df = add_record(scores_df, nested_cv(X, 250, random_seeds, f1_score))
scores_df = add_record(scores_df, nested_cv(X, 250, random_seeds, precision_score))
scores_df = add_record(scores_df, nested_cv(X, 250, random_seeds, recall_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,best overall accuracy,components
0,OneClassSVM,rbf,1.000000e-09,0.13,accuracy_score,nested cv,57.615385,10.810087,64.475000,31.888391,33.428571,25.929260,80.769231,250
1,OneClassSVM,rbf,1.000000e-06,0.36,f1_score,nested cv,61.923077,10.497957,71.031864,22.593188,47.571429,26.481029,80.769231,250
2,OneClassSVM,rbf,1.000000e-08,0.96,precision_score,nested cv,51.923077,8.711347,55.326340,39.215742,18.714286,19.424895,73.076923,250
3,OneClassSVM,rbf,1.000000e-09,0.44,recall_score,nested cv,58.153846,8.724583,66.456128,27.527577,47.142857,34.434202,80.769231,250


In [17]:
scores_df = add_record(scores_df, nested_cv(X, 200, random_seeds, accuracy_score))
scores_df = add_record(scores_df, nested_cv(X, 200, random_seeds, f1_score))
scores_df = add_record(scores_df, nested_cv(X, 200, random_seeds, precision_score))
scores_df = add_record(scores_df, nested_cv(X, 200, random_seeds, recall_score))
scores_df = add_record(scores_df, nested_cv(X, 150, random_seeds, accuracy_score))
scores_df = add_record(scores_df, nested_cv(X, 150, random_seeds, f1_score))
scores_df = add_record(scores_df, nested_cv(X, 150, random_seeds, precision_score))
scores_df = add_record(scores_df, nested_cv(X, 150, random_seeds, recall_score))
scores_df = add_record(scores_df, nested_cv(X, 100, random_seeds, accuracy_score))
scores_df = add_record(scores_df, nested_cv(X, 100, random_seeds, f1_score))
scores_df = add_record(scores_df, nested_cv(X, 100, random_seeds, precision_score))
scores_df = add_record(scores_df, nested_cv(X, 100, random_seeds, recall_score))
scores_df = add_record(scores_df, nested_cv(X, 50, random_seeds, accuracy_score))
scores_df = add_record(scores_df, nested_cv(X, 50, random_seeds, f1_score))
scores_df = add_record(scores_df, nested_cv(X, 50, random_seeds, precision_score))
scores_df = add_record(scores_df, nested_cv(X, 50, random_seeds, recall_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,best overall accuracy,components
0,OneClassSVM,rbf,1.000000e-09,0.13,accuracy_score,nested cv,57.615385,10.810087,64.475000,31.888391,33.428571,25.929260,80.769231,250
1,OneClassSVM,rbf,1.000000e-06,0.36,f1_score,nested cv,61.923077,10.497957,71.031864,22.593188,47.571429,26.481029,80.769231,250
2,OneClassSVM,rbf,1.000000e-08,0.96,precision_score,nested cv,51.923077,8.711347,55.326340,39.215742,18.714286,19.424895,73.076923,250
3,OneClassSVM,rbf,1.000000e-09,0.44,recall_score,nested cv,58.153846,8.724583,66.456128,27.527577,47.142857,34.434202,80.769231,250
4,OneClassSVM,rbf,1.000000e-09,0.15,accuracy_score,nested cv,58.384615,10.594461,62.382262,35.725261,34.285714,29.032002,84.615385,200
5,OneClassSVM,rbf,1.000000e-09,0.82,f1_score,nested cv,62.923077,10.399135,66.462392,25.280907,52.857143,28.284271,80.769231,200
6,OneClassSVM,rbf,1.000000e-09,0.15,precision_score,nested cv,52.230769,8.807272,54.874269,39.757539,18.285714,21.100343,76.923077,200
7,OneClassSVM,rbf,1.000000e-09,0.44,recall_score,nested cv,58.692308,8.730346,73.168445,21.029908,46.571429,33.722759,76.923077,200
8,OneClassSVM,rbf,1.000000e-06,0.21,accuracy_score,nested cv,59.538462,12.504200,64.363869,34.918447,36.714286,31.944148,84.615385,150
9,OneClassSVM,rbf,1.000000e-05,0.24,f1_score,nested cv,65.153846,10.216879,72.189864,19.032992,56.428571,25.724204,84.615385,150


In [18]:
file_path = 'svm_exp2_df.pickle'

with open(file_path, "wb") as file:
    pickle.dump(scores_df, file)